In [ ]:
import csv
import copy
import re
from datetime import datetime, timezone

#import xml.etree.ElementTree as ET
#from lxml import etree

# https://docs.python.org/3/library/xml.dom.minidom.html
# https://docs.python.org/3/library/xml.dom.html
#import xml.dom.minidom as MD

# https://lxml.de/tutorial.html
try:
    from lxml import etree
    from lxml.builder import E
    print("running with lxml.etree")
except ImportError:
    import xml.etree.ElementTree as etree
    print("running with Python's xml.etree.ElementTree")


In [ ]:
XML_DIR = "../../revo-fonto/revo/"
BL_CSV = "../vrt/bonalingvo.csv"

# artikolo traktenda
art_xml = XML_DIR+"zoolog.xml"

In [ ]:
def drv_kap(drv,rad):
    """ elprenas kapvorton el drv, anstataŭigante tld per rad """
    kap = copy.deepcopy(drv.find('kap'))
    tld = kap.find('tld')
    tld.tail = rad + tld.tail if tld.tail else rad
    etree.strip_elements(kap,'tld',with_tail=False)
    print(kap.text)
    return kap.text

def trovu(drvj):
    """ trovas kapvorton en bl_list """
    trovoj = []
    for d in drvj:
        print(d)
        print(etree.tostring(d).decode()[0:100]+"...")
        kap = drv_kap(d,radiko)
        for bl in bl_list:
            if bl[0] == kap:
                trovoj.append(bl)
    return [d,trovoj]

def rim_cnt(vortlist):
    """ konstruas la enhavon de rim-elemento el vortlisto """
    cnt = ["\n    "]
    for text in vortlist:
        if text != '':
            if (text[0] == "'" and text[-1] == "'"):
                cnt.append(E('ke',text[1:-1]))
            else:
                cnt.append(text)
    cnt.append("\n    ")
    cnt.append(E('fnt',E('bib',"BL")))
    cnt.append("\n  ")
    return cnt

def aldonu_rim(drv: etree.Element,csvline: list):
    """ konstruas la rim-elementon kaj aldonas post la lasta snc-elemento de drv-elemento """
    # kreu unue la rimarkon
    # apartigu vortojn kaj interajn signojn, enmetu apostrofojn en la vortojn
    # ĉar 'esprimo' ni transformas al <ke>esprimo</ke>
    rim_txt = re.findall(r"['\w]+|[^'\w]+", csvline[3])
    content = rim_cnt(rim_txt)
    rim = E('rim',*content)
    rim.tail="\n"
    last_snc = drv.findall('snc')[-1]
    if last_snc is not None:
        last_snc.addnext(rim)

def incr_ver(dom: etree.ElementTree):
    """ altigas la eldonnumneron de la artikolo """
    re_mrk = r'\$Id:\s+([^\.]+)\.xml,v\s+(\d)\.(\d+)\s+(?:\d\d\d\d\/\d\d\/\d\d\s+\d\d:\d\d:\d\d)(.*?)\$'
    art = dom.xpath("//art")[0]
    art_mrk = art.get("mrk")
    print(art_mrk)
    match = re.match(re_mrk, art_mrk)
    if match:
        filename = match.group(1)
        major = match.group(2)
        minor = int(match.group(3))+1
        rest =  match.group(4)
        now = datetime.now(timezone.utc)
        tstamp = now.strftime("%Y/%m/%d %H:%M:%S")
        version = f"$Id: {filename}.xml,v {major}.{minor} {tstamp}{rest}$"
        print(version)
        art.set("mrk",version)

In [ ]:
# enlegu la liston bonalingvo

bl_list = []

with open(BL_CSV, mode='r', encoding='utf-8') as file:
    bl_list = list(csv.reader(file))
    print(f"legis {len(bl_list)} linioj el {bl_csv}")
    #for row in bl_list:
    #    print(row)  # Jede Zeile wird als Liste ausgegeben


In [ ]:
# ŝargu XML-artikolon

parser = etree.XMLParser(resolve_entities=False)
DOM = etree.parse(art_xml,parser)

##root = tree.getroot()
##
### Print the root element
##print(f"Root element: {root.tag}")
##
### Iterate through child elements
##for child in root:
##    print(f"Tag: {child.tag}, Attributes: {child.attrib}, Text: {child.text}")

In [ ]:
# ekstraktu radikon kaj kapvortojn de la derivaĵoj
rad = DOM.xpath('//rad')
print(f"# radikoj: {len(rad)}")
radiko = rad[0].text
print(radiko)


drvj = DOM.xpath('//drv')
print(drvj)


In [ ]:
# se ni trovos la vorton en la Bonalingvo-listo
# ni aldonos rimarkon en la fino de la koncerna derivaĵo

trv = trovu(drvj)
print(trv)

In [ ]:
def fix_xml(DOM):
	#re_entity = 
	xml = etree.tostring(DOM, method='xml', xml_declaration=True).decode()
	#xml = '<?xml version="1.0"?>\n<!DOCTYPE vortaro SYSTEM "../dtd/vokoxml.dtd">\n'\
	xml = xml\
		.replace('</snc><rim>','</snc>\n  <rim>')\
		.replace('&Monato;<url ref="/','<url ref="&Monato;/')
	return xml

if trv is not None:
	# kreu unue la rimarkon
	drv = trv[0]
	csvline = trv[1][0]
	aldonu_rim(drv,csvline)
	# altigu la eldonon en art/@mrk
	incr_ver(DOM)	
	print(etree.tostring(DOM).decode())

	# skribu rezultan XML al la dosiero
	print("Reskribu XML al "+art_xml)
	xml = fix_xml(DOM)
	with open(art_xml, "w") as file:
		file.write(xml)
		#DOM.write(file, pretty_print=False, xml_declaration=True, encoding="UTF-8")	